In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots as sbp

In [2]:
df_wsd = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/similarity_wasserstein_geo_isp.csv", index_col = 0)
df_geo_features = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/similarity_grid_geo_features_isp.csv", index_col = 0).T
df_corr = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/similarity_correlation_features_isp.csv", index_col = 0).T

df_zerocell = pd.read_csv(f'/data/housing/data/intermediate/jl_pop_synth/zero_cell_ips.csv', index_col = 0)

df_r2 = pd.read_csv(f'/data/housing/data/intermediate/jl_pop_synth/utility_r2_regression_vs_real_price_isp.csv', index_col = 0)



In [45]:
df_wsd = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/similarity_wasserstein_geo_airbnb.csv", index_col = 0)
df_corr = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/similarity_correlation_features_airbnb.csv", index_col = 0)
df_geo_features = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/similarity_grid_geo_features_airbnb.csv", index_col = 0)

df_zerocell = pd.read_csv(f'/data/housing/data/intermediate/jl_pop_synth/zero_cell_airbnb.csv', index_col = 0)

df_r2 = pd.read_csv(f'/data/housing/data/intermediate/jl_pop_synth/utility_r2_regression_vs_real_price_airbnb.csv', index_col = 0)